In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import pickle
import os

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "fChl"]
data.columns

Index(['Turbidity', 'DO', 'fChl', 'Discharge', 'Height', 'WT', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP',
       'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def svr(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(kernel='rbf'))
    ])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)

    os.makedirs("../models/svr/split", exist_ok=True)
    save_path = f"../models/svr/split/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    y_pred = model.predict(X_test)
    mae = np.around(mean_absolute_error(y_test, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y_test, y_pred), 2)
    r2 = np.around(r2_score(y_test, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y_test), 2)

    print(f"\nPerformance for {target} (Split):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
for target in targets:
    svr(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/svr/split/Turbidity.pkl

Performance for Turbidity (Split):
MAE  = 16.61
RMSE = 28.78
R²   = 17.51 %
MBE  = -9.27
Model saved to ../models/svr/split/DO.pkl

Performance for DO (Split):
MAE  = 0.76
RMSE = 1.0
R²   = 80.29 %
MBE  = -0.49
Model saved to ../models/svr/split/fChl.pkl

Performance for fChl (Split):
MAE  = 2.42
RMSE = 3.15
R²   = 57.21 %
MBE  = -0.73


In [5]:
def cvsvr(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(kernel='rbf'))
    ])

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X, y, cv=kf, n_jobs=-1)
    model.fit(X, y)

    os.makedirs("../models/svr/cross", exist_ok=True)
    save_path = f"../models/svr/cross/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Performance for {target} (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [6]:
for target in targets:
    cvsvr(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/svr/cross/Turbidity.pkl
Performance for Turbidity (5-Fold CV):
MAE  = 18.8
RMSE = 33.0
R²   = 12.1 %
MBE  = -11.96
Model saved to ../models/svr/cross/DO.pkl
Performance for DO (5-Fold CV):
MAE  = 0.75
RMSE = 0.96
R²   = 77.73 %
MBE  = -0.08
Model saved to ../models/svr/cross/fChl.pkl
Performance for fChl (5-Fold CV):
MAE  = 2.2
RMSE = 2.96
R²   = 59.34 %
MBE  = -0.75
